#Table of content
1.   Connect drive
2.   Libraries
3.   Prepare the training data
4.   CNN the classical way
  1.   Data augmentation
  2.   Dealing with an imbalanced data
  3.   In-House Neural Network
5.   Transfer Learning
  1.   Data preparation
  1.   VGG16
  2.   InceptionV3
  3.   Inception model fine tuning : Final Model
6.   Analysis of the result

## Connect drive

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
%cd /gdrive/MyDrive/AN2DL_Challenge1

/gdrive/MyDrive/AN2DL_Challenge1


## Libraries

In [ ]:
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

In [ ]:
import tensorflow as tf
import numpy as np
import os
import random
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix
from PIL import Image

tfk = tf.keras
tfkl = tf.keras.layers
print(tf.__version__)

2.7.0


In [ ]:
# Random seed for reproducibility
seed = 42

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
from collections import Counter

## Prepare the training data

In [ ]:
#!unzip dataset.zip

In [ ]:
source = 'training'

In [ ]:
#the name of the classes
labels = ['Apple','Blueberry','Cherry','Corn','Grape','Orange','Peach','Pepper','Potato','Raspberry','Soybean','Squash','Strawberry','Tomato']

## CNN the classical way

### Data Augmentation

In [ ]:
test_source = 'test'

In [ ]:
# ImageDataGenerator
# ------------------

from tensorflow.keras.preprocessing.image import ImageDataGenerator

apply_data_augmentation = True  

# Create training ImageDataGenerator object
if apply_data_augmentation:
    train_data_gen = ImageDataGenerator(
                                        rotation_range=10,
                                        width_shift_range=10,
                                        height_shift_range=10,
                                        zoom_range=0.3,
                                        horizontal_flip=True,
                                        vertical_flip=False,
                                        fill_mode='constant',
                                        cval=0,
                                        rescale=1./255,
                                        preprocessing_function=None,
                                        data_format='channels_last',
                                        validation_split=0.2,    
                                       )
else:
    train_data_gen = ImageDataGenerator(rescale=1./255)

#here I create the validation data generator
valid_data_gen = ImageDataGenerator(rescale=1./255,validation_split=0.2)

test_data_gen = ImageDataGenerator(rescale=1./255)

In [ ]:
# Create generators to read images from dataset directory
# -------------------------------------------------------

# Batch size
bs = 32

# Image shape
img_h = 256   #height of the images
img_w = 256   #width of the images

num_classes = 14

classes = labels

#Training
train_gen = train_data_gen.flow_from_directory(source,
                                               batch_size=bs,
                                               classes=classes,
                                               class_mode='categorical',
                                               shuffle=True, #it's nice to have them shuffled
                                               subset='training',
                                               seed=seed)  

# Validation 
valid_gen = valid_data_gen.flow_from_directory(source,
                                                batch_size=bs,
                                                classes=classes,
                                                class_mode='categorical',
                                                shuffle=True, 
                                                subset='validation',
                                                seed=seed)

test_gen = test_data_gen.flow_from_directory(test_source, #different from the other
                                                batch_size=1,
                                                classes=classes,
                                                class_mode='categorical',
                                                shuffle=False, #to be able to print the confusion matrix 
                                                subset='training',
                                                seed=seed) # set as validation data


### Dealing with an imbalanced dataset

In [ ]:
# Compute the class weights in order to balance loss during training
counter = Counter(train_gen.classes)                          
max_val = float(max(counter.values()))       
class_weights = {class_id : max_val/num_images for class_id, num_images in counter.items()}   
print(class_weights)

# This will help, conjugated with a slight oversampling

### In-House Neural Network

In [ ]:
input_shape = (256, 256, 3)
epochs = 200

def build_model(input_shape):

    # Build the neural network layer by layer
    input_layer = tfkl.Input(shape=input_shape, name='Input')

    resizing_layer = tfkl.Resizing(64, 64, interpolation="bilinear", crop_to_aspect_ratio=False)(input_layer)

    conv1 = tfkl.Conv2D(
        filters=16,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        #kernel_regularizer = 'l2'
    )(resizing_layer)

    bn_layer1 = tfkl.BatchNormalization()(conv1)

    pool1 = tfkl.MaxPooling2D(
        pool_size = (2, 2)
    )(bn_layer1)

    conv2 = tfkl.Conv2D(
        filters=32,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        #kernel_regularizer = 'l2'
    )(pool1)

    bn_layer2 = tfkl.BatchNormalization()(conv2)


    pool2 = tfkl.MaxPooling2D(
        pool_size = (2, 2)
    )(bn_layer2)

    conv3 = tfkl.Conv2D(
        filters=64,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        #kernel_regularizer = 'l2'
    )(pool2)

    bn_layer3 = tfkl.BatchNormalization()(conv3)

    pool3 = tfkl.MaxPooling2D(
        pool_size = (2, 2)
    )(bn_layer3)

    conv4 = tfkl.Conv2D(
        filters=128,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        #kernel_regularizer = 'l2'
    )(pool3)

    bn_layer4 = tfkl.BatchNormalization()(conv4)

    pool4 = tfkl.MaxPooling2D(
        pool_size = (2, 2)
    )(bn_layer4)

    conv5 = tfkl.Conv2D(
        filters=256,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        #kernel_regularizer = 'l2'
    )(pool4)

    bn_layer5 = tfkl.BatchNormalization()(conv5)

    pool5 = tfkl.MaxPooling2D(
        pool_size = (2, 2)
    )(bn_layer5)

    flattening_layer = tfkl.Flatten(name='Flatten')(pool5)
    flattening_layer = tfkl.Dropout(0.30, seed=seed)(flattening_layer)
    classifier_layer = tfkl.Dense(units=512, name='Classifier', activation='relu')(flattening_layer)
    classifier_layer = tfkl.Dropout(0.30, seed=seed)(classifier_layer)
    output_layer = tfkl.Dense(units=14, activation='softmax', name='Output')(classifier_layer)

    # Connect input and output through the Model class
    model = tfk.Model(inputs=input_layer, outputs=output_layer, name='model')

    # Compile the model
    model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(), metrics='accuracy')

    # Return the model
    return model

In [ ]:
model = build_model(input_shape)
model.summary()

In [ ]:
patience = 15
early_stopping = tfk.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=patience, restore_best_weights=True)

# Train the model
history = model.fit(
    x = train_gen,
    validation_data = valid_gen,
    epochs = 250,
    shuffle = True,
    class_weight = class_weights,
    callbacks = [early_stopping]
)

In [ ]:
from datetime import datetime
now = datetime.now().strftime('%b%d_%H-%M-%S')
# Save model, with the date in its name for no confusion
model.save("model_saved_" + now)

In [ ]:
#In case we want to continue the training of an old model
#model = tfk.models.load_model('D:/ANNDL/model_saved_Nov25_10-01-50')

## Transfer Learning

### Data augmentation

In [ ]:
# ImageDataGenerator

apply_data_augmentation = True  

# Create training ImageDataGenerator object
if apply_data_augmentation:
    train_data_gen = ImageDataGenerator(
                                        rotation_range=10,
                                        width_shift_range=10,
                                        height_shift_range=10,
                                        zoom_range=0.3,
                                        horizontal_flip=True,
                                        vertical_flip=False,
                                        fill_mode='constant',
                                        cval=0,
                                        rescale=1./255,
                                        preprocessing_function=None,
                                        data_format='channels_last',
                                        validation_split=0.2,    
                                       )
else:
    train_data_gen = ImageDataGenerator(rescale=1./255)

#Create the validation data generator
valid_data_gen = ImageDataGenerator(rescale=1./255,validation_split=0.2)

In [ ]:
# Batch size
bs = 32

# Image shape
img_h = 256   #height
img_w = 256   #width

num_classes = 14

classes = labels

#Training
train_gen = train_data_gen.flow_from_directory(source,
                                               batch_size=bs,
                                               classes=classes,
                                               class_mode='categorical',
                                               shuffle=True, #it's nice to have them shuffled
                                               subset='training',
                                               seed=seed)  

# Validation 
valid_gen = valid_data_gen.flow_from_directory(source,
                                                batch_size=bs,
                                                classes=classes,
                                                class_mode='categorical',
                                                shuffle=True, 
                                                subset='validation',
                                                seed=seed)

Found 14189 images belonging to 14 classes.
Found 3539 images belonging to 14 classes.


In [ ]:
# Create Dataset objects
# ----------------------

# Training
train_dataset = tf.data.Dataset.from_generator(lambda: train_gen,
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_h, img_w, 3], [None, num_classes]))

train_dataset = train_dataset.repeat()

# Validation
# ----------
valid_dataset = tf.data.Dataset.from_generator(lambda: valid_gen, 
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_h, img_w, 3], [None, num_classes]))

In [ ]:
# Compute the class weights
counter = Counter(train_gen.classes)                          
max_val = float(max(counter.values()))       
class_weights = {class_id : max_val/num_images for class_id, num_images in counter.items()}   
print(class_weights)

### VGG16

In [ ]:
# VGG16
Vgg16 = tf.keras.applications.VGG16(weights='imagenet', 
                                include_top=False, 
                                input_shape=(img_h, img_w,3))

In [ ]:
# model for Transfer Learning done with VGG16
model = tf.keras.models.Sequential()
model.add(Vgg16)
model.add(tf.keras.layers.GlobalAveragePooling2D())
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Dense(512, activation='relu'))
model.add(tf.keras.layers.Dense(num_classes, activation='softmax'))

In [ ]:
# Loss
loss = tf.keras.losses.CategoricalCrossentropy()

# learning rate
lr = 2e-5
#optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
# Validation metrics
metrics = ['accuracy']
#callbacks
es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5,restore_best_weights=True)
# Compile Model
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [ ]:
# I usually stop the training manually because I don't want colab to crash and lose everything
model.fit(x=train_dataset,
          epochs=30,
          steps_per_epoch=len(train_gen),
          validation_data=valid_dataset,
          validation_steps=len(valid_gen),
          class_weight=class_weights,
          callbacks = es_callback)

In [ ]:
model.save("classification_experiments/vggattempt")

### InceptionV3

In [ ]:
# InceptionV3
inception = tf.keras.applications.InceptionV3(weights='imagenet', 
                                include_top=False, 
                                input_shape=(img_h, img_w,3))

In [ ]:
# model for Transfer Learning done with Inception
model1 = tf.keras.models.Sequential()
model1.add(inception)
model1.add(tf.keras.layers.GlobalAveragePooling2D())
model1.add(tf.keras.layers.Dropout(0.3))
model1.add(tf.keras.layers.Dense(512, activation='relu'))
model1.add(tf.keras.layers.Dense(num_classes, activation='softmax'))

model1.summary()

In [ ]:
# Loss
loss = tf.keras.losses.CategoricalCrossentropy()

# learning rate
lr = 2e-5
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
# -------------------

# Validation metrics
# ------------------

metrics = ['accuracy']
# ------------------

# Compile Model
model1.compile(optimizer=optimizer, loss=loss, metrics=metrics)

#callbacks
es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5,restore_best_weights=True)

In [ ]:
hist1 = model1.fit(x=train_dataset,
          epochs=30,
          steps_per_epoch=len(train_gen),
          validation_data=valid_dataset,
          validation_steps=len(valid_gen),
          class_weight=class_weights,callbacks = es_callback).history

In [ ]:
model1.save("classification_experiments/inceptionattempt")

INFO:tensorflow:Assets written to: classification_experiments/inceptionattempt/assets


### Inception model fine tuning : Final Model

In [ ]:
del model1

In [ ]:
# Re-load the model after transfer learning
ft_model = tfk.models.load_model('classification_experiments/inceptionattempt')
ft_model.summary()

In [ ]:
# Set all Inception layers to True
ft_model.get_layer('inception_v3').trainable = True
for i, layer in enumerate(ft_model.get_layer('inception_v3').layers):
   print(i, layer.name, layer.trainable)

0 input_1 True
1 conv2d True
2 batch_normalization True
3 activation True
4 conv2d_1 True
5 batch_normalization_1 True
6 activation_1 True
7 conv2d_2 True
8 batch_normalization_2 True
9 activation_2 True
10 max_pooling2d True
11 conv2d_3 True
12 batch_normalization_3 True
13 activation_3 True
14 conv2d_4 True
15 batch_normalization_4 True
16 activation_4 True
17 max_pooling2d_1 True
18 conv2d_8 True
19 batch_normalization_8 True
20 activation_8 True
21 conv2d_6 True
22 conv2d_9 True
23 batch_normalization_6 True
24 batch_normalization_9 True
25 activation_6 True
26 activation_9 True
27 average_pooling2d True
28 conv2d_5 True
29 conv2d_7 True
30 conv2d_10 True
31 conv2d_11 True
32 batch_normalization_5 True
33 batch_normalization_7 True
34 batch_normalization_10 True
35 batch_normalization_11 True
36 activation_5 True
37 activation_7 True
38 activation_10 True
39 activation_11 True
40 mixed0 True
41 conv2d_15 True
42 batch_normalization_15 True
43 activation_15 True
44 conv2d_13 True
45

In [ ]:
# Freeze first 285 layers
for i, layer in enumerate(ft_model.get_layer('inception_v3').layers[:285]):
  layer.trainable=False
for i, layer in enumerate(ft_model.get_layer('inception_v3').layers):
   print(i, layer.name, layer.trainable)
ft_model.summary()

0 input_1 False
1 conv2d False
2 batch_normalization False
3 activation False
4 conv2d_1 False
5 batch_normalization_1 False
6 activation_1 False
7 conv2d_2 False
8 batch_normalization_2 False
9 activation_2 False
10 max_pooling2d False
11 conv2d_3 False
12 batch_normalization_3 False
13 activation_3 False
14 conv2d_4 False
15 batch_normalization_4 False
16 activation_4 False
17 max_pooling2d_1 False
18 conv2d_8 False
19 batch_normalization_8 False
20 activation_8 False
21 conv2d_6 False
22 conv2d_9 False
23 batch_normalization_6 False
24 batch_normalization_9 False
25 activation_6 False
26 activation_9 False
27 average_pooling2d False
28 conv2d_5 False
29 conv2d_7 False
30 conv2d_10 False
31 conv2d_11 False
32 batch_normalization_5 False
33 batch_normalization_7 False
34 batch_normalization_10 False
35 batch_normalization_11 False
36 activation_5 False
37 activation_7 False
38 activation_10 False
39 activation_11 False
40 mixed0 False
41 conv2d_15 False
42 batch_normalization_15 False

In [ ]:
ft_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_v3 (Functional)   (None, 6, 6, 2048)        21802784  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dropout (Dropout)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 512)               1049088   
                                                                 
 dense_1 (Dense)             (None, 14)                7182      
                                                                 
Total params: 22,859,054
Trainable params: 3,877,134
Non-trainable params: 18,981,920
____________________________________

In [ ]:
# Compile the model
ft_model.compile(loss=loss, optimizer=optimizer, metrics=metrics)

In [ ]:
# Fine-tune the model
ft_history = ft_model.fit(x=train_dataset,
          epochs=30,
          steps_per_epoch=len(train_gen),
          validation_data=valid_dataset,
          validation_steps=len(valid_gen),
          class_weight=class_weights,callbacks = es_callback).history

Epoch 1/30
444/444 [==============================] - 285s 630ms/step - loss: 0.0067 - accuracy: 0.9990 - val_loss: 0.0403 - val_accuracy: 0.9881
Epoch 2/30
444/444 [==============================] - 286s 646ms/step - loss: 0.0030 - accuracy: 0.9996 - val_loss: 0.0395 - val_accuracy: 0.9876
Epoch 3/30
444/444 [==============================] - 286s 646ms/step - loss: 0.0037 - accuracy: 0.9992 - val_loss: 0.0563 - val_accuracy: 0.9850
Epoch 4/30
444/444 [==============================] - 287s 646ms/step - loss: 0.0019 - accuracy: 0.9995 - val_loss: 0.0386 - val_accuracy: 0.9884
Epoch 5/30
444/444 [==============================] - 285s 643ms/step - loss: 0.0033 - accuracy: 0.9995 - val_loss: 0.0481 - val_accuracy: 0.9881
Epoch 6/30
 51/444 [==>...........................] - ETA: 3:40 - loss: 5.5906e-04 - accuracy: 1.0000

KeyboardInterrupt: ignored

In [ ]:
ft_model.save('classification_experiments/FineTuningModel')

INFO:tensorflow:Assets written to: classification_experiments/FineTuningModel/assets


## Analysis of the result

In [ ]:
# Plot the confusion matrix
from sklearn.metrics import confusion_matrix, top_k_accuracy_score, balanced_accuracy_score

y_pred = model.predict(test_gen)
y_pred_m = np.argmax(y_pred, axis=1)

cm = confusion_matrix(test_gen.classes, y_pred_m, normalize='true')
b_acc = balanced_accuracy_score(test_gen.classes, y_pred_m)

fig, ax = plt.subplots(figsize=(10, 10))
im = ax.imshow(cm)
ax.set_title("Matrice de confusion")
fig.tight_layout()
plt.show()
print('Balanced accuracy = ', b_acc)
